In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.3 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG')

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.fc = torch.nn.Linear(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    total_loss = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def test():
    model.eval()
    correct = 0
    for data in data_loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += pred.eq(data.y).sum().item()
    accuracy = correct / len(dataset)
    return accuracy

for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = test()
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

Processing...
Done!


Epoch: 0, Loss: 0.6359, Test Accuracy: 0.6649
Epoch: 10, Loss: 0.5571, Test Accuracy: 0.7340
Epoch: 20, Loss: 0.5278, Test Accuracy: 0.7340
Epoch: 30, Loss: 0.5110, Test Accuracy: 0.7606
Epoch: 40, Loss: 0.5276, Test Accuracy: 0.7181
Epoch: 50, Loss: 0.5296, Test Accuracy: 0.7553
Epoch: 60, Loss: 0.5224, Test Accuracy: 0.7447
Epoch: 70, Loss: 0.5122, Test Accuracy: 0.7553
Epoch: 80, Loss: 0.5247, Test Accuracy: 0.7553
Epoch: 90, Loss: 0.4888, Test Accuracy: 0.7553
Epoch: 100, Loss: 0.5088, Test Accuracy: 0.7394
Epoch: 110, Loss: 0.4975, Test Accuracy: 0.7447
Epoch: 120, Loss: 0.5167, Test Accuracy: 0.7553
Epoch: 130, Loss: 0.5026, Test Accuracy: 0.7447
Epoch: 140, Loss: 0.5040, Test Accuracy: 0.7553
Epoch: 150, Loss: 0.5104, Test Accuracy: 0.7447
Epoch: 160, Loss: 0.5034, Test Accuracy: 0.7553
Epoch: 170, Loss: 0.5079, Test Accuracy: 0.7553
Epoch: 180, Loss: 0.5080, Test Accuracy: 0.7447
Epoch: 190, Loss: 0.5208, Test Accuracy: 0.7553
